In [1]:
!pip install transformers==4.40.2 datasets evaluate accelerate simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 47.1 

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import nltk
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
nltk.download("movie_reviews")

from collections import defaultdict, Counter
import math
import random

random.seed(0) # Don't change
torch.manual_seed(0)  # Don't change
np.random.seed(0) # Don't change

train_X, train_Y = [], []
test_X, test_Y = [], []

for polarity in movie_reviews.categories():
    label = 0 if polarity == 'neg' else 1
    for fid in movie_reviews.fileids(polarity):
        if random.randrange(5) == 0:
            test_X.append(movie_reviews.raw(fid))
            test_Y.append(label)
        else:
            train_X.append(movie_reviews.raw(fid))
            train_Y.append(label)
print(train_X[0], train_Y[0])

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . 
they seem to have taken this pretty neat concept , but executed it terribly . 
so what are the problems with the movie ? 
well , its main problem is that it's simply too jumbled . 
it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no id

# Assignment III
Doing Assignment III by modifying the following code cell.
Your goal is to train a Transformer based model to achieve an Accuracy of 0.85. The higher, the better.

Your solution should be based on Transformer models.
You are free to adjust [the settings](https://simpletransformers.ai/docs/usage/) of the simpletransformers class and [the pre-trained models](https://huggingface.co/models?sort=trending). You can also build the model with [the transformers package](https://huggingface.co/docs/transformers/en/index) for more control in details, but your model has to follow the output format of [the predict() method](https://simpletransformers.ai/docs/classification-models/) of `simpletransformers` for automatic evaluation.

Please note that your code is expected to be executed in the free version of Colab. So please make sure your model and your training procedure is lightweight enough to be successfully executed by our TAs.

In [3]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True

def build_model(training_instances, training_labels):
  train_data = zip(training_instances, training_labels)
  train_df = pd.DataFrame(train_data)

  model_args = {
    "manual_seed": 0,
    "overwrite_output_dir": True,
    "no_cache": True,
    "save_model_every_epoch": False,
    "save_steps": -1,
    "num_train_epochs": 6,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "learning_rate": 2e-5,
    "use_early_stopping": True,
    "early_stopping_delta": 0.6,
    "early_stopping_patience": 5,
  }

  model = ClassificationModel("roberta", "siebert/sentiment-roberta-large-english", use_cuda=True, args=model_args)

  model.train_model(train_df)
  return model

model = build_model(train_X, train_Y)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

Running Epoch 2 of 6:   0%|          | 0/99 [00:00<?, ?it/s]

Running Epoch 3 of 6:   0%|          | 0/99 [00:00<?, ?it/s]

Running Epoch 4 of 6:   0%|          | 0/99 [00:00<?, ?it/s]

Running Epoch 5 of 6:   0%|          | 0/99 [00:00<?, ?it/s]

Running Epoch 6 of 6:   0%|          | 0/99 [00:00<?, ?it/s]

In [4]:
def predict(model, document):
    return model.predict([document])[0][0]

print(predict(model, "this is an uninteresting movie"))
print(predict(model, "a good movie of this year"))

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

1


## Do Evaluation

In [5]:
correct, total = 0, 0

pred_Y = model.predict(test_X)[0]

for prediction, y in zip(pred_Y, test_Y):
    if prediction == y:
        correct += 1
    total += 1

print("%d / %d = %g" % (correct, total, correct / total))

0it [00:00, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

361 / 422 = 0.85545
